In [ ]:
import pandas as pd

df = pd.read_csv("troop_movements.csv")

counts = df['empire_or_resistance'].value_counts()

# Print the counts
counts_df = counts.reset_index()
counts_df.columns = ['empire_or_resistance', 'count']

# Print the DataFrame
print(counts_df)


In [ ]:
homeworlds = (df['homeworld'].value_counts())
# Print the counts
homeworlds_df = homeworlds.reset_index()
homeworlds_df.columns = ['homeworld', 'count']

# Print the DataFrame
print(homeworlds_df)

In [ ]:
unit_types = (df['unit_type'].value_counts())
# Print the counts
unit_types_df = unit_types.reset_index()
unit_types_df.columns = ['unit_type', 'count']

# Print the DataFrame
print(unit_types_df)

In [ ]:
df['is_resistance'] = df['empire_or_resistance'].apply(lambda x: True if x == "resistance" else False)
print(df)

In [ ]:
from matplotlib import pyplot as plt 

plt.bar(df['empire_or_resistance'].unique(), df['empire_or_resistance'].value_counts())

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#X = df[['homeworld', 'unit_type']]
#print(df.columns)
y = df['is_resistance']


In [ ]:
categorical_features = ['homeworld', 'unit_type']

X = pd.get_dummies(df[['homeworld', 'unit_type']])

In [ ]:
dt = DecisionTreeClassifier()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
importances = dt.feature_importances_



feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': importances})

feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Print the sorted feature importances
print(feature_importances)

# Plot the feature importances
feature_importances.plot(kind='bar', x='Feature', y='Importance', figsize=(8, 6), legend=False)
plt.title("Feature Importance from Decision Tree")
plt.ylabel("Importance Score")
plt.xlabel("Feature")
plt.xticks(rotation=90)
plt.show()


In [ ]:
feature_importances.plot(kind='bar', figsize=(8,6))
plt.title("Feature Importance from Decision Tree")
plt.ylabel("Importance Score")
plt.show()
##plt.bar(feature_importances['Feature'], max(feature_importances['Importance'])) # make names vertica;l
#plt.xticks(rotation=90)

In [ ]:
import pickle as pkl

with open("trained_model.pkl", "wb") as f:
    pkl.dump(dt, f)

In [ ]:
import fastparquet as fp
import pyarrow as pa


df_1m = pd.read_csv("troop_movements_1m.csv")

print(df_1m.head(10))

In [ ]:
#Clean data

df_1m.loc[df_1m["unit_type"] == "invalid_unit", "unit_type"] = "unknown"

df_1m['location_x'] = df_1m['location_x'].bfill()
df_1m['location_y'] = df_1m['location_y'].bfill()

In [ ]:
#check after cleaning data
has_invalid = (df_1m["unit_type"] == "invalid_unit").any()

if has_invalid:
    sample_invalid = df_1m[df_1m["unit_type"] == "invalid_unit"].sample(min(5, len(df_1m[df_1m["unit_type"] == "invalid_unit"])))
    print("Sample of rows with invalid units:")
    print(sample_invalid)
else:
    print("No invalid units found in the DataFrame.")

missing_count = df_1m["location_x"].isna().sum()
total_rows = len(df_1m)
missing_percentage = (missing_count / total_rows) * 100

print(f"Missing location_x values: {missing_count} out of {total_rows} rows ({missing_percentage:.2f}%)")

In [ ]:
fp.write('troop_movements_1m.parquet', df_1m)

with open('trained_model.pkl', 'rb') as f:
    dt = pkl.load(f)

In [ ]:
parquet_file = fp.ParquetFile('troop_movements_1m.parquet')
cleaned_df = parquet_file.to_pandas()


In [ ]:
X = pd.get_dummies(cleaned_df[['homeworld', 'unit_type']])
predictions = dt.predict(X)

print(cleaned_df.head(10))
print(predictions[0 : 10])

In [ ]:
# Add predictions to your DataFrame
cleaned_df['predictions'] = predictions

print(cleaned_df.head(15))